# Tank Classes


This notebook gives a quick overview of the tank classes and their usage.

Tanks are used for both Liquid and Gas storage. Therefore, user may use it
when simulating Liquid or Hybrid propulsion systems.

There are different types of tanks, each with its own characteristics.
Here we will show how to use most of them.
A cylindrical tank will be simulated, but following different approaches to
calculate the propellant mass flow rate.

To summarize, the UllageBasedTank and LevelBasedTank are less accurate than the
MassFlowRateBasedTank and MassBasedTank, since they assume uniform gas
distribution filling all the portion that is not occupied by liquid.
Therefore, these tanks can only model the tank state until the liquid runs out.


In [ ]:
# These lines are here for debugging purposes only.
%load_ext autoreload
%autoreload 2

First, we need to import the necessary classes from the rocketpy module


In [ ]:
from rocketpy import (
    MassFlowRateBasedTank,
    UllageBasedTank,
    LevelBasedTank,
    MassBasedTank,
    Fluid,
    CylindricalTank,
)

## Defining the Fluid's objects


Both liquid and gas phases are defined using the rocketpy.Fluid class.

See the Fluid class documentation for more information.


In [ ]:
LiquidN2O = Fluid(name="Liquid Nitrous Oxide", density=855, quality=1)
VapourN2O = Fluid(name="Vapour Nitrous Oxide", density=101, quality=0)

## Defining the Tank's geometry


As already mentioned, we will use a cylindrical tank for these examples.
This is the simplest tank geometry possible, and it is defined using the
CylindricalTank class.


In [ ]:
tank_geometry = CylindricalTank(
    radius=0.1, height=2.0, spherical_caps=False, geometry_dict=dict()
)

After defining the tank's geometry, we can visualize it to check if it is
correct.


In [ ]:
tank_geometry.radius()

## MassFlowRateBasedTank


In [ ]:
# help(MassFlowRateBasedTank)

In [ ]:
N2O_flow_tank = MassFlowRateBasedTank(
    name="MassFlowRateBasedTank",
    geometry=tank_geometry,
    flux_time=24.750,
    liquid=LiquidN2O,
    gas=VapourN2O,
    initial_liquid_mass=42.8,
    initial_gas_mass=0.1,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out="../../data/motors/liquid_motor_example/liquid_mass_flow_out.csv",
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out="../../data/motors/liquid_motor_example/gas_mass_flow_out.csv",
    discretize=100,
)

In [ ]:
# Evolution of the Propellant Mass and the Mass flow rate
N2O_flow_tank.fluid_mass.plot()
N2O_flow_tank.net_mass_flow_rate.plot()

In [ ]:
# Evolution of the Propellant center of mass position
N2O_flow_tank.center_of_mass.plot()

## UllageBasedTank


First, we calculate the ullage volume.
To do so, we can use the previous tank object.


In [ ]:
tank_volume = tank_geometry.total_volume
ullage = (-1 * N2O_flow_tank.liquid_volume) + tank_volume

In [ ]:
N2O_ullage_tank = UllageBasedTank(
    name="UllageBasedTank",
    geometry=tank_geometry,
    flux_time=24.750,
    ullage=ullage,
    gas=VapourN2O,
    liquid=LiquidN2O,
)

In [ ]:
N2O_ullage_tank.fluid_mass.plot()
N2O_ullage_tank.net_mass_flow_rate.plot()

In [ ]:
N2O_ullage_tank.center_of_mass.plot()

## MassBasedTank


In [ ]:
gas_mass = N2O_flow_tank.gas_mass
liquid_mass = N2O_flow_tank.liquid_mass

In [ ]:
N2O_mass_tank = MassBasedTank(
    name="MassBasedTank",
    geometry=tank_geometry,
    flux_time=24.750,
    gas_mass=gas_mass,
    liquid_mass=liquid_mass,
    gas=VapourN2O,
    liquid=LiquidN2O,
)

In [ ]:
N2O_mass_tank.fluid_mass.plot()
N2O_mass_tank.net_mass_flow_rate.plot()

In [ ]:
N2O_mass_tank.center_of_mass.plot()

## LevelBasedTank


In [ ]:
liquid_height = N2O_flow_tank.liquid_height

In [ ]:
N20_level_tank = LevelBasedTank(
    name="LevelBasedTank",
    geometry=tank_geometry,
    flux_time=24.750,
    liquid=LiquidN2O,
    gas=VapourN2O,
    liquid_height=liquid_height,
    discretize=100,
)

In [ ]:
N20_level_tank.fluid_mass.plot(0, 24.750)
N20_level_tank.net_mass_flow_rate.plot(0, 24.750)

In [ ]:
N20_level_tank.center_of_mass.plot(0, 24.750)

## Compare all the tanks


Now that we saw the different methods to calculate the mass flow rate, we can
compare the results all together.


In [ ]:
from rocketpy import Function

In [ ]:
tanks = [N2O_flow_tank, N2O_ullage_tank, N2O_mass_tank, N20_level_tank]

In [ ]:
# Mass
Function.comparePlots(
    plot_list=[(tank.fluid_mass, tank.name) for tank in tanks],
    lower=0,
    upper=24.750,
    title="Mass of Propellant in the Tank",
    xlabel="Time (s)",
    ylabel="Mass (kg)",
)

In [ ]:
# Mass flow rate
Function.comparePlots(
    plot_list=[(tank.net_mass_flow_rate, tank.name) for tank in tanks],
    lower=0,
    upper=24.750,
    title="Mass Flow Rate Comparison",
    xlabel="Time (s)",
    ylabel="Mass Flow Rate (kg/s)",
)

In [ ]:
# Center of mass
Function.comparePlots(
    plot_list=[(tank.center_of_mass, tank.name) for tank in tanks],
    lower=0,
    upper=24.750,
    title="Center of Mass Comparison",
    xlabel="Time (s)",
    ylabel="Center of mass of Fluid (m)",
)